In [123]:
import pandas as pd
import numpy as np
#import plotly.express as px
from datetime import datetime
import warnings
import json
warnings.filterwarnings("ignore")

In [124]:
sample_tabdata = pd.read_excel('Sample - Superstore (1).xls')
sample_tabdata = sample_tabdata.iloc[:,1:]
newcolmname = sample_tabdata.columns.str.replace(' ', '')
sample_tabdata.columns = list(newcolmname)

In [125]:
#tab_json=sample_tabdata.iloc[:4000,:].to_json(orient ='records')
tab_dict=sample_tabdata.iloc[:4000,:].to_dict()


In [126]:
#sample_tabdata.iloc[:4,:].to_dict()

# Configurations

In [255]:
# Data receviving

_input = {'tabdata':tab_dict,
 'tabparam':{'target_col':'Profit',
            'impcol':['Sales','Quantity'],
            'grp_cols':['Category'],
             'reporting_cols' : ['Region','State'],
             'recency':['date_col',"no_of_point"],
             'corr_colm_check': None
            },
 'tsdata':None,
 'tsparam':{'xaxisname':None,
            'yaxisname': None,
            'impcol':None,
            'grp_col':None,
            'recency': None}  
  }


tab_knowledge_base = ["Max and Min of {0} is {1}. The {0} is correlated with {2} and could be potential reason for max and min.",
                      "Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.", 
                      "Max {self.target_col.lower()} is {self.max} in category {self.max_grp_col}.",
                      "The {0} is correlated with {2} and could be potential reason."]

ts_knowledge_base = ["In last {self.recency} days, peak is {self.peak} {self.yaxisname} on {self.xaxispoint}).", "The {0} is correlated with {2} and could be potential reason for max and min.",
                      "Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.", 
                      "Max {self.target_col.lower()} is {self.max} in category {self.max_grp_col}.",
                      "The {0} is correlated with {2} and could be potential reason."]


In [202]:
if __name__ == '__main__':

    data=Data(_input)
    data.parseinput()
    
    #
    tabinsght = Nlgtabinsight(data.tabdf,data.tabparam)
    tabinsght.getmaxmin()
    

['-1665.1', '-1850.9', '-3840.0']


In [202]:
if __name__ == '__main__':

    data=Data(_input)
    data.parseinput()
    
    #
    tabinsght = Nlgtabinsight(data.tabdf,data.tabparam)
    tabinsght.getmaxmin()
    

['-1665.1', '-1850.9', '-3840.0']


# Data

In [236]:
class Data:
    
    def __init__(self,_input=None):
        self.input = _input
        self.tabdf = None
        self.ts_df = None
        self.tabparam = None
        self.tspram = None
        
    def parseinput(self):
        if self.input['tabdata'] != None:
            #self.tabdf = pd.read_json(self.input['tabdata'])
            self.tabdf = pd.DataFrame(self.input['tabdata'])
            self.tabparam = self.input['tabparam']

        if self.input['tsdata'] != None:    
            self.ts_df = self.input['tsdata']
            self.tsparam = self.input['tsparam']
                

In [237]:
# Tab Insights

In [256]:
class Nlgtabinsight:
    def __init__ (self, df, tab_param ):
        self.df = df
        self.imp_cols = tab_param['impcol']
        self.target_col = tab_param['target_col']
        self.nm =  tab_param['corr_colm_check']
        self.grp_cols = tab_param['grp_cols']
        self.reporting_cols = tab_param['reporting_cols']
        self.corr_val = None
        self.max = None
        self.max_grp_col = None
        self.xaxismax_name = None
        self.min = None
        self.min_grp_col = None
        self.corr_val = None
        self.corr_col_name = None
        
    
#     def read_file(self):
#         df = pd.read_excel('downloads/Sample - Superstore (1).xls')
#         return df
    
    def get_corr(self):
        corr_df = self.df[self.cols]
#         corr_df = df[self.cols + list(target_col)]* We will not need to pass the target col in the col list 
        corr_coeff = pd.DataFrame(corr_df.corr())
        corr_coeff_val = pd.Series(abs(corr_coeff[self.target_col]))
        corr_coeff_tgt = corr_coeff[corr_coeff_val.between(0.3,0.99)]
        corr_val = corr_coeff_tgt[self.target_col].values
        nm = corr_coeff_tgt.columns[corr_coeff_val.between(0.3,0.99)]
#       print(type(corr_coeff_tgt))
        self.nm = nm[0]
        self.corr_val = corr_val[0]
        
    def getmaxmin(self):
        
        df_temp = self.df.copy()
        bottom_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmin(axis=0)]
        self.min = [str(round(x,1)) for x in list(bottom_rows[self.target_col])]
        self.min_grp_col = [str(x) for x in list(bottom_rows[self.grp_cols[0]])]
       
        top_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmax(axis=0)]
        self.max = [str(round(x,1)) for x in list(top_rows[self.target_col])]
        self.max_grp_col = [str(x) for x in list(top_rows[self.grp_cols[0]])]
    
        
        
    def print_val(self):
        if self.nm != None:
            print(f'Corr between {self.nm} and {self.target_col} is {self.corr_val}')
            
    def get_insight(self):
        global tab_knowledge_base
        
        all_insight = []
        all_insight.append(eval(str("f'")+tab_knowledge_base[1]+"'"))
        all_insight.append(eval(str("f'")+tab_knowledge_base[2]+"'"))
        
        #print(f'Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.')
        self.tab_insight = all_insight
        
            
            

# TS insights

In [ ]:
class Nlgtsinsight:
    def __init__ (self, df, ts_param ):
        self.df = df
        self.imp_cols = ts_param['impcol']
        self.target_col = ts_param['xaxisname']
        self.nm =  ts_param['yaxisname']
        self.grp_cols = ts_param['grp_cols']
        self.reporting_cols = ts_param['reporting_cols']
       
        
    
    def get_corr(self):
        corr_df = self.df[self.cols]
#         corr_df = df[self.cols + list(target_col)]* We will not need to pass the target col in the col list 
        corr_coeff = pd.DataFrame(corr_df.corr())
        corr_coeff_val = pd.Series(abs(corr_coeff[self.target_col]))
        corr_coeff_tgt = corr_coeff[corr_coeff_val.between(0.3,0.99)]
        corr_val = corr_coeff_tgt[self.target_col].values
        nm = corr_coeff_tgt.columns[corr_coeff_val.between(0.3,0.99)]
#       print(type(corr_coeff_tgt))
        self.nm = nm[0]
        self.corr_val = corr_val[0]
        
    def getmaxmin(self):
        
        df_temp = self.df.copy()
        bottom_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmin(axis=0)]
        self.min = [str(round(x,1)) for x in list(bottom_rows[self.target_col])]
        self.min_grp_col = [str(x) for x in list(bottom_rows[self.grp_cols[0]])]
       
        top_rows = df_temp.loc[df_temp.groupby(self.grp_cols)[self.target_col].idxmax(axis=0)]
        self.max = [str(round(x,1)) for x in list(top_rows[self.target_col])]
        self.max_grp_col = [str(x) for x in list(top_rows[self.grp_cols[0]])]
    
        
        
    def print_val(self):
        if self.nm != None:
            print(f'Corr between {self.nm} and {self.target_col} is {self.corr_val}')
            
    def get_insight(self):
        global tab_knowledge_base
        
        all_insight = []
        all_insight.append(eval(str("f'")+tab_knowledge_base[1]+"'"))
        all_insight.append(eval(str("f'")+tab_knowledge_base[2]+"'"))
        
        #print(f'Min {self.target_col.lower()} is {self.min} in category {self.min_grp_col}.')
        self.tab_insight = all_insight
        
            
         

In [257]:
if __name__ == '__main__':

    data=Data(_input)
    data.parseinput()
    
    #
    tabinsght = Nlgtabinsight(data.tabdf,data.tabparam)
    temp=tabinsght.getmaxmin()
    
    tabinsght = Nlgtabinsight(data.tsdf,data.tsparam)
    #temp=tabinsght.getmaxmin()
    

In [258]:
tabinsght.get_insight()

In [259]:
tabinsght.tab_insight

["Min profit is ['-1665.1', '-1850.9', '-3840.0'] in category ['Furniture', 'Office Supplies', 'Technology'].",
 "Max profit is ['673.9', '3177.5', '3920.0'] in category ['Furniture', 'Office Supplies', 'Technology']."]

In [146]:
list(temp['Profit'].values)

[-1665.0522, -1850.9464000000007, -3839.9903999999988]

In [106]:
temp.columns = [x[0]+"_"+x[1] for x in temp.columns]


In [107]:
temp.columns
rename(columns={'ID':'COUNT','TIME':'MEAN_TIME'})

NameError: name 'rename' is not defined

In [108]:
temp

,O_r,O_r,S_h,S_h,C_u,C_u,S_e,C_o,C_i,S_t,P_o,R_e,P_r,C_a,S_u,P_r,S_a,Q_u,D_i,P_r
0,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2020-138688,2020-06-12,2020-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,CA-2019-105627,2019-03-08,2019-03-12,Standard Class,DK-12895,Dana Kaydos,Consumer,United States,Kenosha,Wisconsin,53142.0,Central,FUR-CH-10002084,Furniture,Chairs,Hon Mobius Operator's Chair,860.9300,7,0.00,189.4046
3996,CA-2019-105627,2019-03-08,2019-03-12,Standard Class,DK-12895,Dana Kaydos,Consumer,United States,Kenosha,Wisconsin,53142.0,Central,TEC-PH-10003012,Technology,Phones,Nortel Meridian M3904 Professional Digital phone,769.9500,5,0.00,223.2855
3997,CA-2019-105627,2019-03-08,2019-03-12,Standard Class,DK-12895,Dana Kaydos,Consumer,United States,Kenosha,Wisconsin,53142.0,Central,OFF-AR-10002704,Office Supplies,Art,Boston 1900 Electric Pencil Sharpener,14.9800,1,0.00,4.4940
3998,CA-2019-105627,2019-03-08,2019-03-12,Standard Class,DK-12895,Dana Kaydos,Consumer,United States,Kenosha,Wisconsin,53142.0,Central,FUR-FU-10000308,Furniture,Furnishings,Deflect-o Glass Clear Studded Chair Mats,373.0800,6,0.00,82.0776
